In [27]:
from importlib.metadata import files
from pathlib import Path
import os
import numpy as np

In [28]:
def dump(lines, fname, path):
    if fname == 'skipdump':
        return
    with open(path+fname, 'a') as f:
        f.write(''.join(lines))

In [29]:
def getname(fname, phase):
    fsplit = fname.split('_')
    match phase:
        case 1:
            return f"index_{fsplit[-1]}"
        case 2:
            return f"browse_{fsplit[-1]}"

benign_dir = "/proj/spark-cluster-PG0/ransomware-setups/benignware"
in_dir = f"{benign_dir}/media-server/output/syscall_output/"
out_dir = f"{benign_dir}/media-server/output/syscall_output_parsed/"

os.makedirs(out_dir, exist_ok=True)

for file in os.listdir(in_dir):
    print(file)
    with open(in_dir+file, 'r') as f:
        lines = f.readlines()
        phase = 0
        phrases = []
        readout = False
        for l in lines:
            if 'sys_exit_pidfd_getfd' in l:
                if phase > 0:
                    out_fname = getname(file, phase)
                    dump(phrases, out_fname, out_dir)
                    print(f"Wrote {len(phrases)} to {out_fname}")
                readout = True
                phrases = []
                phase+=1
                if phase == 3:
                    break
            elif 'sys_enter_pidfd_open' in l:
                readout = False
            elif readout:
                phrases.append(l)
        if phase == :
            out_fname = getname(file, phase)
            dump(phrases, out_fname, out_dir)
            print(f"Wrote {len(phrases)} to {out_fname}")

SyntaxError: invalid syntax (3532929307.py, line 37)

In [3]:
def get_pname(idx):
    if idx == 1:
        return "asymm"
    if idx == 2:
        return "symm"

def get_count(phasename, path):
    count = 0
    for file in os.listdir(path):
        if phasename in file:
            count+=1
    return (count)

def get_fname(phase, path, original):
    oparts = original.split('_')
    if phase == 'asymm':
        count = get_count(phase, path)
        return f"{phase}_{count}"
    elif phase == 'symm':
        count = get_count(f"{phase}_{oparts[0]}_{oparts[1]}b", path)
        return f"{phase}_{oparts[0]}_{oparts[1]}b_{count}"

def dump_perf_data(data, ptimes, fname, run_iter, outdir):
    # print(f"{fname} {run_iter}")
    #     for key in data.keys():
    #         print(f"{key}: {len(data[key])}")
    processed_data = {}
    for key in data.keys():
        for pt in ptimes:
            if key in pt:
                phases = ptimes[pt]
        pstart = 0
        for i,time in enumerate(phases):
            if i == 0:
                pstart = time
                continue
            pname = get_pname(i)
            # pdata = [{t,data[key][t]} for t in data[key] if t<=time and t>=pstart]
            pdata = {}
            for t in data[key]:
                if t<=time and t>=pstart:
                    pdata.update({t:data[key][t]})
            if pname in processed_data:
                processed_data[pname].update({key:pdata})
            else:
                processed_data[pname] = {key:pdata}
            pstart = time
    print(fname)
    # print(processed_data)
    for ptype in processed_data:
        # ct = get_count(ptype, outdir)
        outfile = get_fname(ptype, outdir, fname)
        tmin = 100
        tmax = 0
        # print(processed_data[ptype])
        header="time,"
        keylist=[]
        for key in processed_data[ptype]:
            # print(key)
            # print(processed_data[ptype][key])
            tmin = list(processed_data[ptype][key].keys())[0] if list(processed_data[ptype][key].keys())[0] < tmin else tmin
            tmax = list(processed_data[ptype][key].keys())[-1] if list(processed_data[ptype][key].keys())[-1] > tmax else tmax
            keylist.append(key)
            header+=f"{key},"
            
        with open(f"{out_dir}/{outfile}","a") as f:
            f.write(header+'\n')
        # print(tmin)
        # print(tmax)
        # print(processed_data[ptype])
        for t in np.arange(tmin,tmax+0.09,0.1):
            t = round(t,1)
            vstring = f"{t},"
            for key in keylist:
                # print(processed_data[ptype][key])
                if t in processed_data[ptype][key].keys():
                    val = processed_data[ptype][key][t]
                else:
                    val = 0
                vstring+=f"{val},"
            with open(f"{out_dir}/{outfile}","a") as f:
                f.write(vstring+'\n')


In [169]:
## Perf counters processor
def save(time, value, metric, data):
    if metric in data.keys():
        data[metric].update({time: value})
    else:
        data.update({metric:{time: value}})
    return data

benign_dir = "/proj/spark-cluster-PG0/ransomware-setups/benignware"
in_dir = f"{benign_dir}/media-server/output/syscall_output/"
out_dir = f"{benign_dir}/media-server/output/syscall_output_parsed/"

os.makedirs(out_dir, exist_ok=True)

# count = 0
# for file in os.listdir(perf_dir):
#     if 'phase' in file and 'exec_' in file:
#         with open(f"{perf_dir}/{file}",'r') as fp:
#             lines = fp.readlines()
#             # print(lines[-3:])
#             if 'str(elapsed_time)\n' in lines[-3:]:
#                 print(file)
#                 count+=1
# print(count)
runs = []
perf_mets = []
for file in os.listdir(in_dir):
    separator = "_"
    file_parts = file.split('_')
    file_run = separator.join(file_parts[1:5])
    if file_run not in runs:
        runs.append(file_run)
    if file[len(file_run)+6:-2] not in perf_mets:
        perf_mets.append(file[len(file_run)+6:-2])
print(runs)
print(perf_mets)
# runs = ['AES_128_O_none']
exit()
all_data = {}
for idx,run in enumerate(runs):
    # print(f"{idx} {run}")
    for i in range(2,5):
        data = {}
        phase_times = {}
        for met in perf_mets:
            file = f"exec_{run}_{met}_{i}"
            phase_file = file[:-2]+'_4_phase'
            with open(f"{perf_dir}/{phase_file}",'r') as fp:
                lines = fp.readlines()[-3:]
                if len(lines) > 3:
                    print(f"Something wrong here:{phase_file}")
                else:
                    p_time = []
                    for l in lines:
                        # p_time.append(math.ceil(float(l[:-2])*10)/10)
                        fl_num = round(float(l[:-2]),2)
                        if (fl_num*10)%1 > 0.4:
                            fl_num = int(fl_num*10+1)/10
                        p_time.append(fl_num)
                    phase_times[met] = p_time
                    # p1_time = round(float(lines[0][:-2]),1)
                    # p2_time = round(float(lines[1][:-2]),1)
                    # p3_time = round(float(lines[2][:-2]),1)
                    # print(f"{p1_time}, {p2_time}, {p3_time}")
                    # print(p_time)
            with open(f"{in_dir}/{file}",'r') as f:
                lines = f.readlines()
                for l in lines[2:]:
                    l=l.replace("<not counted>","0")
                    if "#" in l:
                        continue
                    # print(l)
                    # break
                    parts = l.split()
                    # print(round(float(parts[0]),1))
                    time = round(float(parts[0]),1)
                    # print(time)
                    value = int(parts[1].replace(',',''))
                    metric = parts[2]
                    data = save(time, value, metric, data)
                    # print(data)
        # dump_perf_data(data, phase_times, run, i, out_dir)

['Salsa20_128_WA_none', 'AES_128_O_default', 'Salsa20_128_O_default', 'Salsa20_256_WA_none', 'AES_128_O_none', 'AES_256_WA_default', 'AES_256_O_none', 'AES_256_WA_none', 'Salsa20_256_O_default', 'AES_128_WA_default', 'Salsa20_128_WA_default', 'AES_128_WA_none', 'Salsa20_256_O_none', 'AES_256_O_default', 'Salsa20_128_O_none', 'Salsa20_256_WA_default']
['mem-loads,mem-stores,cache-references,LLC-load-misses', 'uops_executed_port.port_0,uops_executed_port.port_1,uops_executed_port.port_5,uops_executed_port.port_6', 'instructions,br_inst_retired.all_branches,avx_insts.all,block:block_rq_issue', 'uops_executed_port.port_2,uops_executed_port.port_3,uops_executed_port.port_4,uops_executed_port.port_7']


In [178]:
p1count = p2count = p3count = 0

def update_pcounts(fname, path):
    global p1count
    global p2count
    global p3count
    p1count = p2count = p3count = 0
    fsplit = fname.split('_')
    for file in os.listdir(path):
        if 'asymm' in file:
            p1count+=1
        if fsplit[1]+'_'+fsplit[2] in file:
            p2count+=1
        if fsplit[3]+'_'+fsplit[2] in file:
            p3count+=1

def getname(fname, phase):
    global p1count
    global p2count
    global p3count
    fsplit = fname.split('_')
    match phase:
        case 1:
            return f"asymm_{p1count}"
        case 2:
            return f"symm_{fsplit[1]}_{fsplit[2]}b_{p2count}"
        case 3:
            return f"transfer_{fsplit[3]}_{fsplit[2]}t_{p3count}"
    
in_dir = '../outputs/v4_results/out_exec/'
out_dir = '../outputs/v4_results/out_exec_parsed/'

os.makedirs(out_dir, exist_ok=True)

for file in os.listdir(in_dir):
    # print(file)
    with open(in_dir+file, 'r') as f:
        lines = f.readlines()
        update_pcounts(file, out_dir)
        phase = 0
        phrases = []
        readout = False
        for l in lines:
            if ',54321,' in l:
                if 'Port unreachable' not in l:
                    readout = False
                    continue
                if phase > 0:
                    out_fname = getname(file, phase)
                    # print(out_fname)
                    dump(phrases, out_fname, out_dir)
                    # print(f"Wrote {len(phrases)} to {out_fname}")
                readout = True
                phrases = []
                phase+=1
                if phase == 3:
                    break
            elif readout:
                phrases.append(l)
        if phase == 2:
            out_fname = getname(file, phase)
            # print(out_fname)
            dump(phrases, out_fname, out_dir)
            # print(f"Wrote {len(phrases)} to {out_fname}")

In [18]:
def get_pname(idx):
    if idx == 1:
        return "asymm"
    if idx == 2:
        return "symm"

def get_count(phasename, path):
    count = 0
    for file in os.listdir(path):
        if phasename in file:
            count+=1
    return (count)

def get_fname(phase, path, original):
    oparts = original.split('_')
    if phase == 'asymm':
        count = get_count(phase, path)
        return f"{phase}_{count}"
    elif phase == 'symm':
        count = get_count(f"{phase}_{oparts[0]}_{oparts[1]}b", path)
        return f"{phase}_{oparts[0]}_{oparts[1]}b_{count}"

def dump_perf_data(data, ptimes, outfile, outdir):
    processed_data = {}
    if ptimes:
        for key in data.keys():
            for pt in ptimes:
                if key in pt:
                    phases = ptimes[pt]
            pstart = 0
            for i,time in enumerate(phases):
                if i == 0:
                    pstart = time
                    continue
                pname = get_pname(i)
                pdata = {}
                for t in data[key]:
                    if t<=time and t>=pstart:
                        pdata.update({t:data[key][t]})
                if pname in processed_data:
                    processed_data[pname].update({key:pdata})
                else:
                    processed_data[pname] = {key:pdata}
                pstart = time
    else:
        processed_data={"all":data}
    print(outfile)
    for ptype in processed_data:
        # outfile = f"{fname}_{run_iter}"
        tmin = 100
        tmax = 0
        header="time,"
        keylist=[]
        for key in processed_data[ptype]:
            tmin = list(processed_data[ptype][key].keys())[0] if list(processed_data[ptype][key].keys())[0] < tmin else tmin
            tmax = list(processed_data[ptype][key].keys())[-1] if list(processed_data[ptype][key].keys())[-1] > tmax else tmax
            keylist.append(key)
            header+=f"{key},"
            
        with open(f"{out_dir}/{outfile}","a") as f:
            f.write(header+'\n')
        # print(header)
        for t in np.arange(tmin,tmax+0.09,0.1):
            t = round(t,1)
            vstring = f"{t},"
            for key in keylist:
                if t in processed_data[ptype][key].keys():
                    val = processed_data[ptype][key][t]
                else:
                    val = 0
                vstring+=f"{val},"
            with open(f"{out_dir}/{outfile}","a") as f:
                f.write(vstring+'\n')
        # print(vstring+'\n')


In [25]:
## Perf counters processor
def save(time, value, metric, data):
    if metric in data.keys():
        data[metric].update({time: value})
    else:
        data.update({metric:{time: value}})
    return data

# prog_name="browser"
# prog_dir=f"{prog_name}"
# run_idx=-2
# run_count=5

# prog_name="perf"
# prog_dir=f"{prog_name}/cpu2017"
# run_idx=-3
# run_count=3

prog_name="filebench"
prog_dir=f"{prog_name}"
run_idx=-2
run_count=3

benign_dir = "/proj/spark-cluster-PG0/ransomware-setups/benignware"
in_dir = f"{benign_dir}/{prog_dir}/output/hardware_output/"
out_dir = f"{benign_dir}/{prog_dir}/output/hardware_output_parsed/"

os.makedirs(out_dir, exist_ok=True)
phases = False
runs = []
perf_mets = []
file_prefix = f"{prog_name}_hardware"
for file in os.listdir(in_dir):
    separator = "_"
    file_parts = file.split('_')
    file_run = file_parts[run_idx]
    if file_run not in runs:
        runs.append(file_run)
    if separator.join(file_parts[2:run_idx]) not in perf_mets:
        perf_mets.append(separator.join(file_parts[2:run_idx]))
print(runs)
print(perf_mets)

if run_idx == -3:
    add_fname = "_r"
else:
    add_fname = ""

all_data = {}
for idx,run in enumerate(runs):
    for i in range(1,run_count+1):
        data = {}
        phase_times = None
        for met in perf_mets:
            file = f"{file_prefix}_{met}_{run}{add_fname}_{i}"
            if phases:
                phase_times = {}
                phase_file = file[:-2]+'_4_phase'
                with open(f"{perf_dir}/{phase_file}",'r') as fp:
                    lines = fp.readlines()[-3:]
                    if len(lines) > 3:
                        print(f"Something wrong here:{phase_file}")
                    else:
                        p_time = []
                        for l in lines:
                            fl_num = round(float(l[:-2]),2)
                            if (fl_num*10)%1 > 0.4:
                                fl_num = int(fl_num*10+1)/10
                            p_time.append(fl_num)
                        phase_times[met] = p_time
            with open(f"{in_dir}/{file}",'r') as f:
                lines = f.readlines()
                for l in lines[2:]:
                    l=l.replace("<not counted>","0")
                    if "#" in l:
                        continue
                    parts = l.split()
                    time = round(float(parts[0]),1)
                    value = int(parts[1].replace(',',''))
                    metric = parts[2]
                    data = save(time, value, metric, data)
        dump_perf_data(data, phase_times, f"{file_prefix}_{run}_{i}", out_dir)

['varmail', 'randomrw', 'videoserver', 'fileserver', 'oltp']
['uops_executed_port.port_0,uops_executed_port.port_1,uops_executed_port.port_5,uops_executed_port.port_6', 'instructions,br_inst_retired.all_branches,avx_insts.all,block:block_rq_issue', 'uops_executed_port.port_2,uops_executed_port.port_3,uops_executed_port.port_4,uops_executed_port.port_7', 'mem-loads,mem-stores,cache-references,LLC-load-misses']
filebench_hardware_varmail_1
filebench_hardware_varmail_2
filebench_hardware_varmail_3
filebench_hardware_randomrw_1
filebench_hardware_randomrw_2
filebench_hardware_randomrw_3
filebench_hardware_videoserver_1
filebench_hardware_videoserver_2
filebench_hardware_videoserver_3
filebench_hardware_fileserver_1
filebench_hardware_fileserver_2
filebench_hardware_fileserver_3
filebench_hardware_oltp_1
filebench_hardware_oltp_2
filebench_hardware_oltp_3
